In [124]:
import pyodbc
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

query = "SELECT * FROM dbo.Telephonie;"
df_tel = pd.read_sql(query, cnxn)

query = "SELECT * FROM dbo.Eptica;"
df_ept = pd.read_sql(query, cnxn)

/var/folders/gv/tfv3kbl11y58yd1f4j17_fqr0000gn/T/ipykernel_3222/2178636582.py:15: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

/var/folders/gv/tfv3kbl11y58yd1f4j17_fqr0000gn/T/ipykernel_3222/2178636582.py:18: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [89]:
df_tel['date_appel'] = pd.to_datetime(df_tel['date_appel']) # On encode la column "date_appel" pour l'exploiter en timeseries
df_tel['nombre_entrants_corrige'] = df_tel['nombre_entrants_corrige'].astype(int) # On transforme les float en Int car un demi appel n'existe pas
df_tel.head()

,id,date_appel,entite,famille,nombre_entrants_corrige
0,12,2019-01-02,Entite 1,F2,165
1,13,2019-01-02,Entite 1,F3,54
2,14,2019-01-02,Entite 1,F4,1
3,15,2019-01-02,Entite 1,F5,99
4,16,2019-01-02,Entite 1,F6,1487


In [90]:
# Nombre d'entités (5)
df_tel['entite'].unique()

array(['Entite 1', 'Entite 2', 'Entite 3', 'Entite 4', 'Entite 5'],
      dtype=object)

In [91]:
# Nombre de famille (7)
df_tel['famille'].unique()

array(['F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F1'], dtype=object)

In [125]:
# On définit nos dates comme index
df_tel = df_tel.set_index(df_tel['date_appel'].rename("date")).drop("date_appel", axis=1)

# Renome la columns appels reçus
df_tel.rename(columns={"nombre_entrants_corrige":"nb_recus"}, inplace=True)

In [121]:
import plotly.express as px

#
# Création de graphique dynamique de toutes les Entités avec toutes les Familles
#


for entite in df_tel.entite.unique():
    df = df_tel.loc[df_tel.entite == entite].groupby([pd.Grouper(freq = 'w'), "famille"]).agg(nb_recus = ('nb_recus' , 'sum')).reset_index().set_index("date")
    fig = px.line(df, x=df.index, y="nb_recus", color='famille',
              title=f"Nombre d'appel pour {entite}")
    fig.update_xaxes(
        dtick="M1",
        tickformat="%b\n%Y",
        ticklabelmode="period")
    fig.show()